<a href="https://colab.research.google.com/github/i-huzaifa-arshad/InvokeAi-Colab/blob/Alpha/InvokeAI_Colab_v3_6_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
<div style="text-align: center">
    <h1></h1>
</div>
<h1><center>Colab Notebook for InvokeAi v3.6.2</center></h1>

<h3><center>Enjoy the latest version of InvokeAI 😉</center></h3>

<center>
    <a href="https://www.patreon.com/huzaifaarshad" title="My Patreon"><img src="https://upload.wikimedia.org/wikipedia/commons/5/5a/Patreon_logomark.svg" width="20" height="20"></a>
   &nbsp;
    <a href="https://github.com/i-huzaifa-arshad" title="My GitHub"><img src="https://github.githubassets.com/images/modules/logos_page/GitHub-Mark.png" width="20" height="20"></a>
</center>



---


In [ ]:
#@markdown # **1. Connect Google Drive**
#@markdown ---

#@markdown - If you have low storage in drive, skip this step and proceed to install InvokeAi in Colab Storage
#@markdown - Please note once you restart runtime, you will lose all data, so download important files before closing the notebook

from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
#@markdown # **2. Install / Update InvokeAI**
#@markdown ---
#@markdown - Running this code will automatically create `InvokeAI-v3.6.2` folder and clone the repository in it, so no need to create a separate folder

from IPython.display import clear_output

%cd /content/gdrive/MyDrive

# Clone the repository
!git clone https://github.com/i-huzaifa-arshad/InvokeAI-v3.6.2
clear_output()

print('\u2714 Done!')

In [ ]:
#@markdown # **3. Install Dependencies (Required)**
#@markdown ---
#@markdown - It takes about ~8-10 minutes to install the required dependencies
#@markdown - In case you think it's not working, don't worry. Sit back and relax 😉

import warnings
warnings.filterwarnings("ignore")

%cd /content/gdrive/MyDrive/InvokeAI-v3.6.2

# # Add the directory of your cloned repository to sys.path
import sys
sys.path.append('/content/gdrive/MyDrive/InvokeAI-v3.6.2')

print('[1;32mInstalling dependencies...')

# Installing requirements from Dependencies folder
!pip install --upgrade pip setuptools > nul 2>&1
!pip install --use-pep517 --upgrade --force-reinstall -r Dependencies/Requirements.txt > nul 2>&1
!pip install -q -e . > nul 2>&1

print ('\u2714 Done')

In [ ]:
#@markdown # **4-a. Model Download**
#@markdown ---

#@markdown - Installing stable diffusion model (SD 1.5). If you want to install ControlNet and/or other models (SD XL, etc), click on the below path
#@markdown - `/content/gdrive/MyDrive/InvokeAI-v3.6.2/invokeai/configs/INITIAL_MODELS.yaml` and change `recommended: False` to `recommended: True`
#@markdown - Installing SD 1.5 model will take about ~3-5 minutes. If you selected other models as well, it will take more time due to their size (or your internet speed)
#@markdown - If you get `AttributeError: 'set' object has no attribute 'extend'` error, just delete the `GUI` folder from your drive and run the cell again

import os
import warnings

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.filterwarnings("ignore")

%cd /content/gdrive/MyDrive/InvokeAI-v3.6.2

print('[1;32mDownloading model(s)...')

!python /content/gdrive/MyDrive/InvokeAI-v3.6.2/scripts/invokeai-model-install.py --root_dir /content/gdrive/MyDrive/InvokeAI-v3.6.2/GUI --yes > /dev/null 2>&1

print ('\u2714 Done')


In [ ]:
#@markdown # **4-b. Download Custom model (Optional)**
#@markdown ---

#@markdown - Download a trained dreambooth `.ckpt model` or other desired models from `google drive` or `civitai.com`
#@markdown - Just write desired `model name` and `model url` and it will automatically download the models in the required path
#@markdown - You don't have to write model extension in `model_name` as it will automatically add it and save models like `model-name.ckpt` or `model-name.safetensors`
#@markdown - If you get some error about model while running the InvokeAI, install the model using `model manager` inside InvokeAI Webui

from IPython.display import clear_output
import os
from urllib.parse import urlparse

# Ask the user for the model name
model_name = "" # @param {type:"string"}

model_link = "" # @param {type:"string"}

# Define the directory where the model will be saved
model_dir = '/content/gdrive/MyDrive/InvokeAI-v3.6.2/GUI/models/sd-1/main'

# Create the directory if it does not exist
os.makedirs(model_dir, exist_ok=True)

# Check if the URL is from Google Drive
if "drive.google.com" in model_link:
    # Add the .ckpt extension
    model_filename = f"{model_name}.ckpt"

    # Extract the file ID from the URL
    file_id = model_link.split('/')[-2]

    # Download the file from Google Drive
    !gdown https://drive.google.com/uc?id={file_id} -O {os.path.join(model_dir, model_filename)}

else:
    # Add the .safetensors extension
    model_filename = f"{model_name}.safetensors"

    # Download the model
    !wget -O {os.path.join(model_dir, model_filename)} "{model_link}"

clear_output()
print('Model Downloaded! 🚀')


In [ ]:
#@markdown # **5. Run InvokeAI**
#@markdown - Launching InvokeAI Web UI via SSH Tunnel
#@markdown - The outputs will be saved in the `Outputs` folder under `/content/gdrive/MyDrive/InvokeAI-v3.6.2/GUI` path
#@markdown - Open this first link ending with `.moe` to access the InvokeAI WebUI, once you see the this kind of url `http://127.0.0.1:9090` at the bottom
#@markdown ---

import os
import shlex
import subprocess
from IPython.display import clear_output
from pathlib import Path
from typing import Union
clear_output()


import warnings
warnings.filterwarnings("ignore")

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

id_rsa_file = "/root/.ssh/id_rsa"
id_rsa_pub_file = "/root/.ssh/id_rsa.pub"
if os.path.exists(id_rsa_file):
    os.remove(id_rsa_file)
if os.path.exists(id_rsa_pub_file):
    os.remove(id_rsa_pub_file)
clear_output()

def gen_key(path: Union[str, Path]) -> None:
    path = Path(path)
    arg_string = f'ssh-keygen -t rsa -b 4096 -N "" -q -f {path.as_posix()}'
    args = shlex.split(arg_string)
    subprocess.run(args, check=True)
    path.chmod(0o600)

ssh_name = "id_rsa"
ssh_path = Path("/root/.ssh/") / ssh_name
gen_key(ssh_path)
clear_output()

import threading
def tunnel():
  !ssh -R 80:127.0.0.1:9090 -o StrictHostKeyChecking=no -i /root/.ssh/id_rsa remote.moe
threading.Thread(target=tunnel, daemon=True).start()

%cd /content/gdrive/MyDrive/InvokeAI-v3.6.2/
!python /content/gdrive/MyDrive/InvokeAI-v3.6.2/scripts/invokeai-web.py --root /content/gdrive/MyDrive/InvokeAI-v3.6.2/GUI